In [3]:
from gelos.config import RAW_DATA_DIR, PROJ_ROOT, PROCESSED_DATA_DIR, DATA_VERSION, REPORTS_DIR, FIGURES_DIR
import geopandas as gpd
import yaml

In [10]:

output_dir = PROCESSED_DATA_DIR / DATA_VERSION
data_root = RAW_DATA_DIR / DATA_VERSION
chip_gdf = gpd.read_file(output_dir / 'gelos_chip_tracker_with_tsne.geojson')
reports_dir = REPORTS_DIR / DATA_VERSION
reports_dir.mkdir(exist_ok=True, parents=True)
figures_dir = FIGURES_DIR / DATA_VERSION
figures_dir.mkdir(exist_ok=True, parents=True)

yaml_config_directory = PROJ_ROOT / 'gelos' / 'configs'
for yaml_filepath in yaml_config_directory.glob("*.yaml"):
    with open(yaml_filepath, "r") as f:
        yaml_config = yaml.safe_load(f)
    print(yaml.dump(yaml_config))
    model_name = yaml_config['model']['init_args']['model']
    model_title = yaml_config['model']['title']
    embedding_extraction_strategies = yaml_config['embedding_extraction_strategies']

    # add variables to yaml config so it can be passed to classes
    yaml_config['data']['init_args']['data_root'] = data_root
    yaml_config['model']['init_args']['output_dir'] = output_dir
    embeddings_directories = [item for item in output_dir.iterdir() if item.is_dir()]


data:
  class_path: gelos.gelosdatamodule.GELOSDataModule
  init_args:
    bands:
      DEM:
      - DEM
      S1RTC:
      - VV
      - VH
      S2L2A:
      - COASTAL_AEROSOL
      - BLUE
      - GREEN
      - RED
      - RED_EDGE_1
      - RED_EDGE_2
      - RED_EDGE_3
      - NIR_BROAD
      - NIR_NARROW
      - WATER_VAPOR
      - SWIR_1
      - SWIR_2
    batch_size: 1
    num_workers: 0
    repeat_bands:
      DEM: 4
    target_size: 96
embedding_extraction_strategies:
  All Patches from April to June:
  - start: 1
    step: 1
    stop: 2
  All Steps of Middle Patch:
  - start: 0
    step: 1
    stop: null
  - start: 18
    step: 1
    stop: 19
model:
  class_path: terratorch.tasks.EmbeddingGenerationTask
  init_args:
    embed_file_key: filename
    embedding_pooling: null
    has_cls: false
    layers:
    - -1
    model: terramind_v1_base
    model_args:
      merge_method: mean
      modalities:
      - S2L2A
      - S1RTC
      - DEM
      pretrained: true
    output_format

In [18]:

for embeddings_directory in embeddings_directories:

    embedding_layer = embeddings_directory.stem

    for extraction_strategy, slice_args in embedding_extraction_strategies.items():
        print(extraction_strategy, model_title)
        break


CLS Token Prithvi EO V2 300M
CLS Token Prithvi EO V2 300M
